In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [15]:
city_data_df = pd.read_csv('weather_data/cities.csv')
# city_data_df.head()
city_data_df["Max Temp"]

0      69.91
1      60.01
2      -3.59
3      71.01
4      73.40
       ...  
555    59.83
556    82.40
557    79.88
558    28.31
559    66.99
Name: Max Temp, Length: 560, dtype: float64

In [4]:
city_data_df.dtypes

City ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [19]:
# Configure gmaps to use API key from config
gmaps.configure(api_key=g_key)

In [20]:
max_temp = city_data_df["Max Temp"]
temps=[]
for temp in max_temp:
    temps.append(max(temp,0))

In [23]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300,point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat","Lng"]]
clouds = city_data_df["Cloudiness"]
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds,dissipating=False, max_intensity=300,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
# Heatmap of wind speed
locations = city_data_df[["Lat","Lng"]]
wind = city_data_df["Wind Speed"]
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind,dissipating=False, max_intensity=300,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [37]:
# filter through dataframe to find cities that fit criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Barbate,ES,2020-04-17 15:02:30,36.19,-5.92,71.01,68,75,8.05
4,4,Kapaa,US,2020-04-17 15:02:30,22.08,-159.32,73.40,78,1,11.41
10,10,Bredasdorp,ZA,2020-04-17 15:02:34,-34.53,20.04,71.60,35,96,13.87
12,12,Vila Velha,BR,2020-04-17 15:01:46,-20.33,-40.29,82.99,69,75,13.87
14,14,Vaini,TO,2020-04-17 15:02:35,-21.20,-175.20,73.40,88,20,4.70
22,22,Mwingi,KE,2020-04-17 15:02:37,-0.93,38.06,74.75,63,96,7.18
24,24,Uarini,BR,2020-04-17 15:02:38,-2.99,-65.11,75.20,100,90,2.24
26,26,Viedma,AR,2020-04-17 15:02:38,-40.81,-63.00,76.91,36,65,18.32
30,30,Alyangula,AU,2020-04-17 15:02:39,-13.85,136.42,75.20,100,97,6.31
31,31,Pisco,PE,2020-04-17 15:02:39,-13.70,-76.22,73.99,30,51,8.05


In [38]:
preferred_cities_df.count()

City ID       216
City          216
Country       216
Date          216
Lat           216
Lng           216
Max Temp      216
Humidity      216
Cloudiness    216
Wind Speed    216
dtype: int64

In [39]:
hotel_df = preferred_cities_df[["City","Country","Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Barbate,ES,71.01,36.19,-5.92,
4,Kapaa,US,73.40,22.08,-159.32,
10,Bredasdorp,ZA,71.60,-34.53,20.04,
12,Vila Velha,BR,82.99,-20.33,-40.29,
14,Vaini,TO,73.40,-21.20,-175.20,
22,Mwingi,KE,74.75,-0.93,38.06,
24,Uarini,BR,75.20,-2.99,-65.11,
26,Viedma,AR,76.91,-40.81,-63.00,
30,Alyangula,AU,75.20,-13.85,136.42,
31,Pisco,PE,73.99,-13.70,-76.22,


In [40]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [42]:
for index, row in hotel_df.iterrows():
    # Get lat and long from record
    lat = row["Lat"]
    lng = row["Lng"]
    
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
     # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [44]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Barbate,ES,71.01,36.19,-5.92,Hotel Adiafa
4,Kapaa,US,73.40,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
10,Bredasdorp,ZA,71.60,-34.53,20.04,Bredasdorp Country Manor
12,Vila Velha,BR,82.99,-20.33,-40.29,Hotel Vitória Palace
14,Vaini,TO,73.40,-21.20,-175.20,Keleti Beach Resort
...,...,...,...,...,...,...
545,Bubaque,GW,82.17,11.28,-15.83,Casa Dora
546,Sao Geraldo Do Araguaia,BR,84.06,-6.40,-48.55,Hotel Rio Araguaia
551,Metro,ID,76.89,-5.11,105.31,Hotel Familie2
556,Iranshahr,IR,82.40,27.20,60.68,کافی نت دانشگاه


In [45]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [49]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [51]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))